In [1]:
import pandas as pd

In [8]:
train=pd.read_csv("/Users/fan/Downloads/train.csv", encoding='latin-1')

In [9]:
train.head()

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...


In [16]:
train = train.rename(index=str, columns={"SentimentText": "tweet"})

In [17]:
train.head()

,ItemID,Sentiment,tweet
0,1,0,is so sad for my APL frie...
1,2,0,I missed the New Moon trail...
2,3,1,omg its already 7:30 :O
3,4,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,5,0,i think mi bf is cheating on me!!! ...


## 1. Basic Feature Extraction

In [18]:
# word count
train['word_count']=train['tweet'].apply(lambda x:len(str(x).split(" ")))
train[['tweet','word_count']].head()

,tweet,word_count
0,is so sad for my APL frie...,28
1,I missed the New Moon trail...,25
2,omg its already 7:30 :O,19
3,.. Omgaga. Im sooo im gunna CRy. I'...,36
4,i think mi bf is cheating on me!!! ...,24


In [19]:
# the number of characters
train['char_count']=train['tweet'].str.len()
train[['tweet','char_count']].head()

,tweet,char_count
0,is so sad for my APL frie...,61
1,I missed the New Moon trail...,51
2,omg its already 7:30 :O,37
3,.. Omgaga. Im sooo im gunna CRy. I'...,132
4,i think mi bf is cheating on me!!! ...,53


In [20]:
# avg lenth of each word
def avg_word(sentence):
    words=sentence.split()
    return (sum(len(word) for word in words)/len(words))

train['avg_word']=train['tweet'].apply(lambda x:avg_word(x))
train[['tweet','avg_word']].head()

,tweet,avg_word
0,is so sad for my APL frie...,4.857143
1,I missed the New Moon trail...,4.500000
2,omg its already 7:30 :O,3.800000
3,.. Omgaga. Im sooo im gunna CRy. I'...,3.880000
4,i think mi bf is cheating on me!!! ...,3.333333


In [21]:
# the number of stop words
from nltk.corpus import stopwords

stop=stopwords.words('english')
train['stopwords']=train['tweet'].apply(lambda sen:len([x for x in sen.split() if x in stop]))
train[['tweet','stopwords']].head()

,tweet,stopwords
0,is so sad for my APL frie...,4
1,I missed the New Moon trail...,1
2,omg its already 7:30 :O,1
3,.. Omgaga. Im sooo im gunna CRy. I'...,7
4,i think mi bf is cheating on me!!! ...,3


In [28]:
# hashtag and special characters
train['hashtags']=train['tweet'].apply(lambda sen:len([x for x in sen.split() if (x.startswith("#") or x.startswith("@"))]))
train[['tweet','hashtags']].head()

,tweet,hashtags
0,is so sad for my APL frie...,0
1,I missed the New Moon trail...,0
2,omg its already 7:30 :O,0
3,.. Omgaga. Im sooo im gunna CRy. I'...,0
4,i think mi bf is cheating on me!!! ...,0


In [27]:
# the number of numeric numbers
train['numerics']=train['tweet'].apply(lambda sen:len([x for x in sen.split() if x.isdigit()]))
train[['tweet','numerics']].head()

,tweet,numerics
0,is so sad for my APL frie...,0
1,I missed the New Moon trail...,0
2,omg its already 7:30 :O,0
3,.. Omgaga. Im sooo im gunna CRy. I'...,1
4,i think mi bf is cheating on me!!! ...,0


In [29]:
# the number of words with upper case
train['upper']=train['tweet'].apply(lambda sen:len([x for x in sen.split() if x.isupper()]))
train[['tweet','upper']].head()

,tweet,upper
0,is so sad for my APL frie...,1
1,I missed the New Moon trail...,1
2,omg its already 7:30 :O,1
3,.. Omgaga. Im sooo im gunna CRy. I'...,1
4,i think mi bf is cheating on me!!! ...,1


## 2. Pre-processing

In [30]:
# turn all upper case to lower case
train['tweet']=train['tweet'].apply(lambda sen:" ".join(x.lower() for x in sen.split()))
train['tweet'].head()

0             is so sad for my apl friend.............
1                     i missed the new moon trailer...
2                              omg its already 7:30 :o
3    .. omgaga. im sooo im gunna cry. i've been at ...
4               i think mi bf is cheating on me!!! t_t
Name: tweet, dtype: object

In [31]:
# punctuation removal
train['tweet'] = train['tweet'].str.replace('[^\w\s]','')
train['tweet'].head()

0                          is so sad for my apl friend
1                        i missed the new moon trailer
2                                omg its already 730 o
3     omgaga im sooo im gunna cry ive been at this ...
4                  i think mi bf is cheating on me t_t
Name: tweet, dtype: object

In [32]:
# stopwords removal
train['tweet']=train['tweet'].apply(lambda sen:" ".join(x for x in sen.split() if x not in stop))
train['tweet'].head()

0                                       sad apl friend
1                              missed new moon trailer
2                                      omg already 730
3    omgaga im sooo im gunna cry ive dentist since ...
4                             think mi bf cheating t_t
Name: tweet, dtype: object

In [33]:
# frequent words removal
freq=pd.Series(' '.join(train['tweet']).split()).value_counts()[:10]
freq

im        11010
good       5790
like       5473
get        5272
lol        5043
u          5021
dont       4794
know       4529
love       4403
thanks     4032
dtype: int64

In [34]:
freq=list(freq.index)
freq

['im', 'good', 'like', 'get', 'lol', 'u', 'dont', 'know', 'love', 'thanks']

In [35]:
train['tweet']=train['tweet'].apply(lambda sen:' '.join(x for x in sen.split() if x not in freq))
train['tweet'].head()

0                                       sad apl friend
1                              missed new moon trailer
2                                      omg already 730
3    omgaga sooo gunna cry ive dentist since 11 sup...
4                             think mi bf cheating t_t
Name: tweet, dtype: object

In [36]:
# rare(infrequent) words removal
infreq = pd.Series(' '.join(train['tweet']).split()).value_counts()[-10:]
infreq = list(infreq.index)
train['tweet'] = train['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in infreq))
train['tweet'].head()

0                                       sad apl friend
1                              missed new moon trailer
2                                      omg already 730
3    omgaga sooo gunna cry ive dentist since 11 sup...
4                             think mi bf cheating t_t
Name: tweet, dtype: object